In [30]:
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
url = 'https://raw.githubusercontent.com/tomwhite/covid-19-uk-data/master/data/covid-19-cases-uk.csv'
df = pd.read_csv(url, error_bad_lines=False)


In [32]:
#format as date
df['Date']=pd.to_datetime(df.Date)
#sort by date
df.sort_values(by='Date')

#change '1 to 4' to 2
df.TotalCases = pd.to_numeric(df.TotalCases, errors='coerce').fillna(2).astype(np.int64)

#filter out awaiting clarification areas
df=df[df.Area!='awaiting clarification']
df=df[df.Area!='Awaiting confirmation']



In [33]:
#filter out outliers
df=df.drop(df[(df['Area']=='Sheffield')&(df['Date']==datetime.datetime(2020, 3, 17))].index)
df=df.drop(df[(df['Area']=='Derbyshire')&(df['Date']==datetime.datetime(2020, 3, 17))].index)

In [35]:
#create dictionary for areas with more than 10 cases
areaWithMoreThan10={}


for index, row in df.iterrows():
    if row.TotalCases>10:
#         print(row.Area,row.Date)
        if row.Area not in areaWithMoreThan10.keys():
            areaWithMoreThan10[row.Area] = {'Area':row.Area,"Code":row.AreaCode,"firstDate":row.Date, "casesOnDay0":row.TotalCases}


In [36]:
areaWithMoreThan10

{'Devon': {'Area': 'Devon',
  'Code': 'E10000008',
  'firstDate': Timestamp('2020-03-08 00:00:00'),
  'casesOnDay0': 12},
 'Hertfordshire': {'Area': 'Hertfordshire',
  'Code': 'E10000015',
  'firstDate': Timestamp('2020-03-08 00:00:00'),
  'casesOnDay0': 13},
 'Kensington and Chelsea': {'Area': 'Kensington and Chelsea',
  'Code': 'E09000020',
  'firstDate': Timestamp('2020-03-10 00:00:00'),
  'casesOnDay0': 13},
 'Hampshire': {'Area': 'Hampshire',
  'Code': 'E10000014',
  'firstDate': Timestamp('2020-03-11 00:00:00'),
  'casesOnDay0': 13},
 'Lothian': {'Area': 'Lothian',
  'Code': 'S08000024',
  'firstDate': Timestamp('2020-03-12 00:00:00'),
  'casesOnDay0': 11},
 'Barnet': {'Area': 'Barnet',
  'Code': 'E09000003',
  'firstDate': Timestamp('2020-03-13 00:00:00'),
  'casesOnDay0': 14},
 'Camden': {'Area': 'Camden',
  'Code': 'E09000007',
  'firstDate': Timestamp('2020-03-13 00:00:00'),
  'casesOnDay0': 13},
 'Kent': {'Area': 'Kent',
  'Code': 'E10000016',
  'firstDate': Timestamp('2020-

In [37]:
#Work out time since 10 cases
theseAreas=[]
diffInCases=[]

for index, row in df.iterrows():
    if row.Area in areaWithMoreThan10.keys():
#         print(row.Date-areaWithMoreThan10[row.Area]['firstDate'])
        if row.Date-areaWithMoreThan10[row.Area]['firstDate'] >= datetime.timedelta(days = 0):
            theseAreas.append(row.Date-areaWithMoreThan10[row.Area]['firstDate'])
            diffInCases.append(row.TotalCases-areaWithMoreThan10[row.Area]['casesOnDay0'])
        else:
            theseAreas.append(None)
            diffInCases.append(None)
    else:
        theseAreas.append(None)
        diffInCases.append(None)

df['timeSince10']=theseAreas   
df['diffInCases']=diffInCases

In [38]:
filterArea=df[~pd.isnull(df.timeSince10)]
#just get data where cases have timeSince10

In [42]:
lines=filterArea.pivot(index='Area',values='diffInCases',columns='timeSince10').transpose()

In [12]:
#lines.plot(figsize=[12,13],yscale='log')
#ah balls, just export it and do it in datawrapper

In [44]:
#save as .csv
lines.to_csv('jbm2 '+str(datetime.date.today())+'.csv', index = True)

In [45]:
type(lines)

pandas.core.frame.DataFrame